In [ ]:
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
import torch  # assuming PyTorch for YOLO
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolo11n.pt") 


# Set up video capture and output
input_video = 'easy_9.mp4'
output_video = 'output_video.mp4'
cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Tracking variables
frame_id = 0
tracks = {}  # Dictionary to store tracked individuals
person_count = 0
id_counter = 0  # Counter to assign new IDs
colors = {}  # To store colors for each ID

def generate_color(id):
    # Generate a unique color for each ID
    np.random.seed(id)
    return tuple(np.random.randint(0, 255, 3).tolist())

def compute_iou(bbox1, bbox2):
    # Compute Intersection over Union (IoU) between two bounding boxes
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    xi1, yi1 = max(x1, x2), max(y1, y2)
    xi2, yi2 = min(x1 + w1, x2 + w2), min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    bbox1_area = w1 * h1
    bbox2_area = w2 * h2
    union_area = bbox1_area + bbox2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

def compute_cost_matrix(detections, previous_detections):
    # Create a cost matrix using IoU between previous and current detections
    cost_matrix = np.zeros((len(previous_detections), len(detections)))
    for i, prev_det in enumerate(previous_detections):
        for j, det in enumerate(detections):
            cost_matrix[i, j] = 1 - compute_iou(prev_det['bbox'], det['bbox'])  # Use 1 - IoU as the cost
    return cost_matrix

def update_tracks(tracks, detections, previous_detections):
    global id_counter
    # Update tracks using Hungarian Algorithm for association
    if len(previous_detections) == 0:
        # First frame, assign new IDs to all detections
        for det in detections:
            det_id = id_counter
            tracks[det_id] = {'bbox': det['bbox'], 'color': generate_color(det_id)}
            colors[det_id] = tracks[det_id]['color']
            det['id'] = det_id
            id_counter += 1
    else:
        # Compute cost matrix and apply Hungarian algorithm for matching
        cost_matrix = compute_cost_matrix(detections, previous_detections)
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        assigned = set()
        
        # Update matched detections
        for r, c in zip(row_ind, col_ind):
            if cost_matrix[r, c] < 0.5:  # Match only if cost (1 - IoU) is below threshold
                prev_det = previous_detections[r]
                det = detections[c]
                det_id = prev_det['id']
                tracks[det_id]['bbox'] = det['bbox']
                det['id'] = det_id
                assigned.add(c)
        
        # Assign new IDs to unmatched detections
        for i, det in enumerate(detections):
            if i not in assigned:
                det_id = id_counter
                tracks[det_id] = {'bbox': det['bbox'], 'color': generate_color(det_id)}
                colors[det_id] = tracks[det_id]['color']
                det['id'] = det_id
                id_counter += 1

def draw_bounding_box(frame, bbox, color, det_id):
    # Draw bounding box and ID on frame
    x, y, w, h = bbox
    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
    cv2.putText(frame, f'ID: {det_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

previous_detections = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Step 1: Perform detection
    results = model(frame)  # Perform YOLOv11 detection

    # Extract bounding boxes for people (class index 0)
    detections = results[0].numpy() if isinstance(results[0], torch.Tensor) else results[0]

    people_detections = []
    for det in detections:
        if len(det) >= 6:
            x1, y1, x2, y2, confidence, class_id = det[:6]
            if int(class_id) == 0:  # Only 'person' class
                x, y, w, h = int(x1), int(y1), int(x2 - x1), int(y2 - y1)
                people_detections.append({
                    'bbox': (x, y, w, h),
                    'confidence': confidence
                })
    
    # Step 2: Update tracks using Hungarian algorithm
    update_tracks(tracks, people_detections, previous_detections)
    previous_detections = people_detections  # Store for next frame matching

    # Update the person count
    person_count = len(tracks)

    # Step 3: Draw results on frame
    for det in people_detections:
        draw_bounding_box(frame, det['bbox'], colors[det['id']], det['id'])
    cv2.putText(frame, f'People Count: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Save the processed frame to the output video
    out.write(frame)
    frame_id += 1

# Release resources
cap.release()
out.release()
print("Tracking complete. Video saved to", output_video)


0: 384x640 7 persons, 2 cars, 1 truck, 25.0ms
Speed: 3.0ms preprocess, 25.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


ValueError: not enough values to unpack (expected 6, got 1)